## AMATH Scientific Computing
### Homework-5
#### Manjaree Binjolkar

In [1]:
import numpy as np
import numpy.matlib
import scipy.integrate
import scipy.sparse
import scipy.sparse.linalg
import mpl_toolkits.mplot3d
from scipy.fft import fft, ifft, fft2, ifft2
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import copy

In [2]:

def cheb(N):
    # N is the number of points in the interior.
    if N==0:
        D = 0
        x = 1
        return D, x
    vals = np.linspace(0, N, N+1)
    x = np.cos(np.pi*vals/N)
    x = x.reshape(-1, 1)
    c = np.ones(N-1)
    c = np.pad(c, (1,), constant_values = 2)
    c *= (-1)**vals
    c = c.reshape(-1, 1)
    X = np.tile(x, (1, N+1))
    dX = X-X.T                  
    D  = (c*(1/c.T))/(dX+(np.eye(N+1)))       #off-diagonal entries
    D  = D - np.diag(sum(D.T))                #diagonal entries

    return D, x


## Problem 1
In both of our solves we will use x,y ∈[−10,10], β = 1, D1 = D2 = 0.1, and will solve
for t ∈ [0,25] with a step size of ∆t = 0.5.

In [3]:
tspan = np.arange(0,25.5,0.5)

#Define the drag term
nu = 0.001
L=20
n=64

beta = 1
D1 = 0.1
D2 = 0.1

#Setup our x and y domain
x2 = np.linspace(-L/2,L/2,n+1) 
x=x2[0:-1] 
y=x.copy()

#Setup the x values.
r1=np.arange(0,n/2,1)
r2=np.arange(-n/2,0,1)
kx = (2*np.pi/L)*np.concatenate((r1,r2))
ky=kx.copy()

#Put X and Y on a meshgrid
[X,Y]=np.meshgrid(x,y)

#Do the same for the k values
[KX,KY]=np.meshgrid(kx,ky)

#This term shows up in the Laplacian 
#-->don't need this since I am directly using KX and KY
#K = KX**2+KY**2
#Kvec = K.reshape(n**2)#take transpose in function

In [4]:
X, Y = np.meshgrid(x,y)
m = 3
alpha = 0

#initial conditions
u = (np.tanh(np.sqrt(X**2+Y**2))-alpha)*np.cos(m*np.angle(X+1j*Y) - np.sqrt(X**2+Y**2))
v = (np.tanh(np.sqrt(X**2+Y**2))-alpha)*np.sin(m*np.angle(X+1j*Y) - np.sqrt(X**2+Y**2))

In [5]:
A1 = copy.deepcopy(X)
#print(np.shape(A1))

In [6]:
A2 = copy.deepcopy(u)
#print(np.shape(A2))

In [7]:
# initial conditions in fourier space, vector form
U_hat_0 = fft2(u)
V_hat_0 = fft2(v)   

#converting to a vector because to send to the solver
U_hat_0_vec = U_hat_0.reshape(n**2,order= 'F')
V_hat_0_vec = V_hat_0.reshape(n**2,order= 'F')

In [8]:
A3 = copy.deepcopy(np.real(U_hat_0))
#print(np.shape(A3))
A3

array([[ -0.12412672,  26.9682989 ,  14.43354622, ...,  13.87596015,
         14.43354622,  26.9682989 ],
       [-27.46083307,  -0.36840745, -12.90316012, ..., -13.4607462 ,
        -12.90316012,  -0.36840745],
       [-11.90256638,  15.18985924,   2.65510656, ...,   2.09752049,
          2.65510656,  15.18985924],
       ...,
       [-23.91541405,   3.17701157,  -9.35774111, ...,  -9.91532718,
         -9.35774111,   3.17701157],
       [-11.90256638,  15.18985924,   2.65510656, ...,   2.09752049,
          2.65510656,  15.18985924],
       [-27.46083307,  -0.36840745, -12.90316012, ..., -13.4607462 ,
        -12.90316012,  -0.36840745]])

In [9]:
init_vec = np.concatenate((U_hat_0_vec,V_hat_0_vec),axis=0).squeeze()
#print(np.shape(init_vec))

In [10]:
#print(np.shape(init_vec[4096:8192]))

In [11]:
#init_vec[4096:8192].reshape(n,n)

In [12]:
A4 = copy.deepcopy(np.imag(init_vec.reshape(8192,1)))
#print(np.shape(A4))
A4

array([[  -0.        ],
       [-111.49057098],
       [-330.59935067],
       ...,
       [ 321.75185555],
       [  41.27229164],
       [ 184.35039871]])

In [13]:
def vortrhs(t, vec, D1, D2, beta, n, KX, KY):
    # Reshape the Fourier-transformed vector
    # into an nxn matrix
    u_double_hat = vec[0:4096].reshape(n, n,order= 'F')
    v_double_hat = vec[4096:8192].reshape(n, n, order= 'F')
    
    #Write all of the terms in the physical space
    u = np.real(ifft2(u_double_hat))
    v = np.real(ifft2(v_double_hat))
    #need fourier transformed data--->u_double_hat
    
    #calculate a, lambda, omega (vectors)
    A_2 = u**2 + v**2
    lambda_A = 1 - A_2
    omega_A = -beta*A_2
    
    #print("A_2", np.shape(A_2))
    #print("lambda_A", np.shape(lambda_A))
    #print("omega_A", np.shape(omega_A))
    
    #calculate u_t and v_t
    # Ut= λ(A)U −ω(A)V + D1∇2U
    # Vt= ω(A)U −λ(A)V + D2∇2V
    
    NL_u = lambda_A*u - omega_A*v 
    NL_v = omega_A*u - lambda_A*v
    
    #print("NL_u", np.shape(NL_u))
    #print("NL_u", np.shape(NL_u))
    
    u_t = fft2(NL_u) + D1*(-(KX**2)*u_double_hat-(KY**2)*u_double_hat)
    v_t = fft2(NL_v) + D2*(-(KX**2)*v_double_hat-(KY**2)*v_double_hat)
    
    #print("u_t", np.shape(u_t))
    #print("v_t", np.shape(v_t))
    
    #Convert everything back and return a vector in the Fourier space.
    u_t_col = u_t.reshape(n**2,order= 'F')
    v_t_col = v_t.reshape(n**2,order= 'F')
    
    #print("u_t_col", np.shape(u_t_col))
    #print("v_t_col", np.shape(v_t_col))
    
    rhs = np.concatenate([u_t_col,v_t_col],axis=0).squeeze()
    #print(np.shape(rhs))

    return rhs

In [14]:
#np.shape(init_vec)
print(tspan)

[ 0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.   6.5
  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12.  12.5 13.  13.5
 14.  14.5 15.  15.5 16.  16.5 17.  17.5 18.  18.5 19.  19.5 20.  20.5
 21.  21.5 22.  22.5 23.  23.5 24.  24.5 25. ]


In [15]:
# integrate in fourier space
sol = scipy.integrate.solve_ivp(lambda t,wfvec: vortrhs(t, wfvec, D1, D2, beta, n, KX, KY),[0, 25], init_vec, t_eval = tspan)

In [16]:
np.shape(sol.y)

(8192, 51)

In [17]:
A5 = copy.deepcopy(np.real(sol.y))
print(np.shape(A5))
A5

(8192, 51)


array([[-1.24126724e-01, -1.85208355e+01, -3.11817424e+01, ...,
        -2.42441169e+02, -2.50412704e+02, -2.56144931e+02],
       [-2.74608331e+01, -3.17619542e+01, -3.04323090e+01, ...,
        -1.16066504e+02, -1.14903372e+02, -1.12664907e+02],
       [-1.19025664e+01, -2.19687373e+01, -2.61782512e+01, ...,
        -1.64414601e+02, -1.68672003e+02, -1.71219459e+02],
       ...,
       [-9.27811132e-01, -1.23172730e+00, -1.03434845e+00, ...,
         7.69393799e+00,  8.81455819e+00,  1.02414147e+01],
       [-3.70225060e-01, -6.19817732e+00, -9.90800907e+00, ...,
        -6.83103873e+01, -7.11280094e+01, -7.38190476e+01],
       [ 1.21645276e+01,  1.07291449e+01,  7.10099808e+00, ...,
        -6.60098363e+01, -6.48644565e+01, -6.31065240e+01]])

In [18]:
A6 = copy.deepcopy(np.imag(sol.y))
print(np.shape(A6))
A6

(8192, 51)


array([[   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [-111.49057098, -177.32775258, -204.48900484, ..., -241.45793839,
        -244.34339926, -244.92362837],
       [-330.59935067, -296.77553603, -221.98873733, ...,  -18.83974865,
         -22.35360836,  -27.83580515],
       ...,
       [ 321.75185555,  352.7669839 ,  289.8010318 , ..., -164.77191812,
        -166.94081843, -169.37217462],
       [  41.27229164,   86.29871441,  103.27255962, ...,  -13.32661587,
         -14.97455832,  -18.03840731],
       [ 184.35039871,  189.71866513,  145.5301011 , ..., -172.92086912,
        -179.90198774, -188.2975376 ]])

In [19]:
sol.y[0,1]

(-18.520835548547815+0j)

In [20]:
#np.shape(sol.y[:,5])
#t=2 on the 5th

In [21]:
#np.shape(np.real(sol.y[0:4096,5])) 

In [22]:
#np.sqrt(4096)

In [23]:
A7 = copy.deepcopy(np.real(sol.y[0:4096,4]).reshape(4096,1))
#print(np.shape(A7))
#print(A7)

# u vector is in 0 to 4096
A8 = copy.deepcopy((np.real(sol.y[0:4096,4])).reshape(64,64).T)
#print(np.shape(A8))
#print(A8)

A9 = copy.deepcopy(np.real(ifft2(sol.y[0:4096,4].reshape(64,64).T)))
#print(np.shape(A9))
A9

array([[-0.3016566 , -0.19954917, -0.04481181, ..., -0.33599327,
        -0.35352404, -0.34852306],
       [-0.64475668, -0.61703499, -0.48148686, ..., -0.42962951,
        -0.50767665, -0.59386354],
       [-0.75494739, -0.7907254 , -0.71091835, ..., -0.4634504 ,
        -0.538604  , -0.65222852],
       ...,
       [ 0.72251944,  0.64867299,  0.57679747, ...,  0.33743702,
         0.57662119,  0.71663886],
       [ 0.59397034,  0.57665859,  0.53791175, ...,  0.17705862,
         0.3806944 ,  0.53564253],
       [ 0.22879916,  0.30558584,  0.36576119, ..., -0.10327906,
         0.00321206,  0.12456776]])

In [24]:
#print(tspan[4])

In [25]:
#plt.scatter(x, y, c=z, cmap='jet',vmin=0, vmax=250)
#plt.colorbar()
#plt.show()

In [26]:
#print(A9)
#np.real(sol.y[0:4096,5]).reshape(64,64)
#print(np.shape(tspan))
#print(np.shape(np.real(sol.y[0:4096,50])))
#print(np.shape(X))
#print(np.shape(Y))

In [27]:
#np.random.seed(19680801)
#Z = np.real(sol.y[0:4096,50])

#fig, ax = plt.subplots()
#ax.pcolormesh(X, Y, Z)

In [28]:
#np.sqrt(4096)

In [29]:
# Generate data for the plot
#[X,Y]=np.meshgrid(x,y)
#tspan[50] =25
#z = np.real(ifft2(sol.y[0:4096,50].reshape(64,64)))
#print(np.shape(z))

# Generate the plot
#fig, ax = plt.subplots()
#cmap = ax.pcolor(X, Y, z.T)
#fig.colorbar(cmap)
#plt.show(fig)

## Problem 2

Dirichlet Boundary conditions,
U(x = −10,y,t) = U(x = 10,y,t) = U(x,y = −10,t) = U(x,y = 10,t) = 0,
using the Chebyshev pseudo-spectral method. To do so, set m = 2, α = 1, and use
n = 30 when creating the Chebyshev matrix and Chebyshev points. You should
use the functions I provided for creating the matrix and points.

In [30]:
# differentiation cheb matrix +b.c.

# We want to solve with 20 interior points. 
##N = 20
N = 30


# Create Chebyshev matrix and Chebyshev grid.
# D is an (N+1)x(N+1) matrix.
[D, x] = cheb(N)
#x = x.reshape(N+1) #get the x values from Chebyshev function

L = 20
# Rescale x
x = x*L/2 #, x goes from -10 to +10

# Derivative operator squared is second derivative
D2 = D@D

# Apply BCs
# Remove the first and last row and column
D2 = D2[1:-1, 1:-1]
#print(np.shape(D2))
# Make sure x changes accordingly too
x2 = x[1:-1]
y2 = x2.copy()
[X,Y]=np.meshgrid(x2,y2)

# This rescales the derivative
D2 = 4/(L**2)*D2 #---->how do we get this?

#tspan = np.arange(0,1+0.05,0.05)
tspan = np.arange(0,25+0.5,0.5)

#print(np.shape(tspan))
#print(np.shape(x2))
#print(np.shape(y2))
#print(np.shape(D2))

In [31]:
# Explore two different initial conditions.
#Uinit = np.exp(-(X**2+Y**2)/0.1);
#print(np.shape(Uinit))
# Uinit = np.exp(-(X**2+Y**2)/0.1)*np.cos((X-1)*5)*np.cos((Y-1)*5)

#reshape to a vector, for solving
#uinitvec = Uinit.reshape((N-1)**2)
#print(np.shape(uinitvec))

In [32]:
# Explore two different initial conditions.
m2 = 2
alpha2 = 1

#initial conditions
u = (np.tanh(np.sqrt(X**2+Y**2))-alpha2)*np.cos(m2*np.angle(X+1j*Y) - np.sqrt(X**2+Y**2))
v = (np.tanh(np.sqrt(X**2+Y**2))-alpha2)*np.sin(m2*np.angle(X+1j*Y) - np.sqrt(X**2+Y**2))
#print(np.shape(u))
#print(np.shape(v))

In [33]:
u = u.T.reshape((N-1)**2)
v = v.T.reshape((N-1)**2)

In [34]:
U_V_init = np.concatenate([u,v])
#u_v_initvec = U_V_init.T.reshape((N-1)**2)

In [35]:
I = np.eye(len(D2))
#print(np.shape(I))
# laplacian matrix. Use "kron" to do that. Don't need to worry so much
# about how it works.
Lap = np.kron(D2,I)+np.kron(I,D2);
#print(np.shape(Lap))

In [36]:
A10 = copy.deepcopy(Lap)
print(np.shape(A10))
A10

(841, 841)


array([[-716.05830164,  130.18235497,  -31.52748086, ...,   -0.        ,
           0.        ,   -0.        ],
       [  60.77492019, -432.7042812 ,   35.60324311, ...,    0.        ,
          -0.        ,    0.        ],
       [  -8.29488678,   23.57433588, -390.50730788, ...,   -0.        ,
           0.        ,   -0.        ],
       ...,
       [  -0.        ,    0.        ,   -0.        , ..., -390.50730788,
          23.57433588,   -8.29488678],
       [   0.        ,   -0.        ,    0.        , ...,   35.60324311,
        -432.7042812 ,   60.77492019],
       [  -0.        ,    0.        ,   -0.        , ...,  -31.52748086,
         130.18235497, -716.05830164]])

In [37]:
A11 = copy.deepcopy(Y)
print(np.shape(A11))
A11

(29, 29)


array([[ 9.94521895e+00,  9.94521895e+00,  9.94521895e+00,
         9.94521895e+00,  9.94521895e+00,  9.94521895e+00,
         9.94521895e+00,  9.94521895e+00,  9.94521895e+00,
         9.94521895e+00,  9.94521895e+00,  9.94521895e+00,
         9.94521895e+00,  9.94521895e+00,  9.94521895e+00,
         9.94521895e+00,  9.94521895e+00,  9.94521895e+00,
         9.94521895e+00,  9.94521895e+00,  9.94521895e+00,
         9.94521895e+00,  9.94521895e+00,  9.94521895e+00,
         9.94521895e+00,  9.94521895e+00,  9.94521895e+00,
         9.94521895e+00,  9.94521895e+00],
       [ 9.78147601e+00,  9.78147601e+00,  9.78147601e+00,
         9.78147601e+00,  9.78147601e+00,  9.78147601e+00,
         9.78147601e+00,  9.78147601e+00,  9.78147601e+00,
         9.78147601e+00,  9.78147601e+00,  9.78147601e+00,
         9.78147601e+00,  9.78147601e+00,  9.78147601e+00,
         9.78147601e+00,  9.78147601e+00,  9.78147601e+00,
         9.78147601e+00,  9.78147601e+00,  9.78147601e+00,
         9.78

In [38]:
A12 = copy.deepcopy(v.reshape(29,29).T)
print(np.shape(A12))
A12

(29, 29)


array([[-8.80245680e-14, -3.10614757e-13, -9.12114426e-13,
        -2.45159271e-12, -6.22035607e-12, -1.46219260e-11,
        -2.98022489e-11, -4.40895348e-11, -9.89405616e-12,
         2.06910901e-10,  8.03932229e-10,  1.79983571e-09,
         2.72814948e-09,  2.91276347e-09,  2.28722315e-09,
         1.47446754e-09,  9.50480702e-10,  6.79340950e-10,
         4.80701506e-10,  3.00137666e-10,  1.61244014e-10,
         7.60929311e-11,  3.26145855e-11,  1.31251499e-11,
         5.09660552e-12,  1.94232478e-12,  7.26761333e-13,
         2.60704730e-13,  8.80245680e-14],
       [-2.60704730e-13, -5.78124628e-13, -1.40936871e-12,
        -3.46983254e-12, -8.35475962e-12, -1.88219825e-11,
        -3.63761283e-11, -4.73787435e-11,  1.77401614e-11,
         3.41234410e-10,  1.16547620e-09,  2.44127432e-09,
         3.45560302e-09,  3.34186127e-09,  2.22843399e-09,
         1.13128217e-09,  6.41871594e-10,  5.39914068e-10,
         4.67224335e-10,  3.31685037e-10,  1.92544665e-10,
         9.57

In [39]:
A13 = copy.deepcopy(U_V_init.reshape(1682,1))
print(np.shape(A13))
print(A13)

(1682, 1)
[[-1.21194666e-12]
 [-1.50784863e-12]
 [-2.10977018e-12]
 ...
 [-7.26761333e-13]
 [-2.60704730e-13]
 [-8.80245680e-14]]


In [40]:
D21 = 0.1
D22 = 0.1

where A2 = U2 + V 2 and ∇2 = ∂2x+ ∂2y. We will consider a particular system with
λ(A) = 1 −A2 (3)
ω(A) = −βA2,

In [41]:
def rhs_heat_cheb(t, init_vec, beta, Lap, D21, D22, N):
    #This is really simple now, just like finite difference
    #breakdown u to two separate vectors coressponding to U and V
    # into an nxn matrix
    u = init_vec[0:841]
    v = init_vec[841:]
    #print("u", np.shape(u))
    #print("v", np.shape(u))
    #print("Lap", np.shape(Lap))
    
    #calculate a, lambda, omega (vectors)
    A_2 = u**2 + v**2
    lambda_A = 1 - A_2
    omega_A = -beta*A_2
    
    #print("A_2", np.shape(A_2))
    #print("lambda_A", np.shape(lambda_A))
    #print("omega_A", np.shape(omega_A))
    
    #calculate u_t and v_t
    # Ut= λ(A)U −ω(A)V + D1∇2U
    # Vt= ω(A)U −λ(A)V + D2∇2V
    u_t = lambda_A*u - omega_A*v + D21*Lap@u
    #print("u_t", np.shape(u_t))
    v_t = omega_A*u - lambda_A*v + D22*Lap@v
    #print("v_t", np.shape(v_t))
    
    #concatenate u_t and v_t
    rhs = np.concatenate([u_t,v_t])
    
    return rhs

In [42]:
sol2 = scipy.integrate.solve_ivp(lambda t,uvec: rhs_heat_cheb(t, uvec, beta, Lap, D21, D22, N), [0, 25+0.5], U_V_init, t_eval = tspan)
#ysol2=sol2.y

In [43]:
#print(np.shape(sol2.y))
#sol2

In [44]:
A14 = copy.deepcopy(sol2.y.T)
print(np.shape(A14))
A14

(51, 1682)


array([[-1.21194666e-12, -1.50784863e-12, -2.10977018e-12, ...,
        -7.26761333e-13, -2.60704730e-13, -8.80245680e-14],
       [-8.06411146e-07,  3.14848380e-07, -5.41917397e-07, ...,
        -8.05888371e-07, -1.35011948e-06,  7.82566320e-06],
       [-6.19956359e-07,  1.25451170e-07, -5.52713406e-07, ...,
        -2.62154959e-07, -5.72987347e-07,  2.77351526e-06],
       ...,
       [ 8.47347310e-03,  3.22928274e-02,  6.26878113e-02, ...,
        -1.03373542e-02, -4.26603326e-03, -1.02930387e-03],
       [ 8.44166946e-03,  3.21897420e-02,  6.25046684e-02, ...,
        -1.02884108e-02, -4.25738709e-03, -1.01043745e-03],
       [ 8.42340230e-03,  3.21419011e-02,  6.24345455e-02, ...,
        -1.02411693e-02, -4.23473586e-03, -1.00798528e-03]])

In [51]:
#tspan
print(A14[0,-1])

-8.802456802288632e-14


In [49]:
A15 = copy.deepcopy(sol2.y[841:1682,4].reshape(841,1))
print(np.shape(A15))
A15

(841, 1)


array([[-1.35078430e-05],
       [ 2.67961645e-06],
       [-4.85482821e-07],
       [ 1.02226848e-07],
       [-3.37328000e-08],
       [ 1.47985216e-08],
       [-1.57272942e-08],
       [ 2.45094364e-08],
       [-3.96051976e-08],
       [ 6.31613172e-08],
       [-9.30974913e-08],
       [ 2.63937043e-07],
       [ 2.39445657e-06],
       [ 3.47127483e-05],
       [ 2.39791789e-05],
       [ 2.99822508e-05],
       [ 1.97929741e-06],
       [ 2.72808618e-07],
       [-1.05722463e-07],
       [ 6.85583220e-08],
       [-4.31027235e-08],
       [ 2.69124894e-08],
       [-1.75290214e-08],
       [ 1.62332953e-08],
       [-3.49092396e-08],
       [ 1.03172640e-07],
       [-4.85931773e-07],
       [ 2.67718508e-06],
       [-1.34937053e-05],
       [ 2.67971733e-06],
       [-5.29421996e-07],
       [ 7.97687116e-08],
       [-8.06062880e-09],
       [-9.47376382e-09],
       [ 1.57867719e-08],
       [-2.01776630e-08],
       [ 2.51462762e-08],
       [-3.24508155e-08],
       [ 4.4

In [50]:
A16 = np.pad(A15.reshape(29,29).T,[1,1])
np.shape(A16)
A16

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00, -1.35078430e-05,  2.67971733e-06,
        -4.85856742e-07,  1.02740221e-07, -3.46536700e-08,
         1.63161179e-08, -1.83079527e-08,  2.90822110e-08,
        -4.82437142e-08,  8.09758176e-08, -1.48211658e-07,
         1.62006588e-07, -2.83484045e-06, -1.28544621e-05,
        -6.12567264e-05, -1.75896864e-05, -3.25166009e-06,
         1.71341829e-07, -1.60

In [48]:
# Generate data for the plot

# Create the grid again
#y = x.copy()
#[X,Y]=np.meshgrid(x2,y2)

# t=25 for v vector 
#z4 = np.real(sol2.y[841:1682,50]).reshape(29,29)
#print(np.shape(z4))

# Generate the plot
#fig, ax = plt.subplots()
#cmap = ax.pcolor(X, Y, z4.T)
#fig.colorbar(cmap)
#plt.show(fig)